In [ ]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
from misc_functions import load_sig_chans, channel_names_to_indices

import pandas as pd
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

### make subjects rois to electrodes dict. Don't need to run this more than once.

In [ ]:
# Initialize the outer dictionary.
subjects_electrodestoROIs_dict = {}

In [ ]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

# subjects = ['D0103'] #testing cuz d0065 being weird

for sub in subjects:
    # sub = 'D0059'
    task = 'GlobalLocal'
    output_name = "Response_fixationCrossBase_1sec_mirror"
    events = ["Response"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False


    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    default_dict = gen_labels(good.info)
    
    # Create rawROI_dict for the subject
    rawROI_dict = defaultdict(list)
    for key, value in default_dict.items():
        rawROI_dict[value].append(key)
    rawROI_dict = dict(rawROI_dict)

    # Filter out keys containing "White-Matter"
    filtROI_dict = {key: value for key, value in rawROI_dict.items() if "White-Matter" not in key}

    # Store the dictionaries in the subjects dictionary
    subjects_electrodestoROIs_dict[sub] = {
        'default_dict': dict(default_dict),
        'rawROI_dict': dict(rawROI_dict),
        'filtROI_dict': dict(filtROI_dict)
    }


# # Save to a JSON file. Uncomment when actually running.
filename = 'subjects_electrodestoROIs_dict.json'
with open(filename, 'w') as file:
    json.dump(subjects_electrodestoROIs_dict, file, indent=4)

print(f"Saved subjects_dict to {filename}")

In [ ]:
# Load from a JSON file
filename = 'subjects_electrodestoROIs_dict.json'

with open(filename, 'r') as file:
    subjects_electrodestoROIs_dict = json.load(file)

print(f"Loaded data from {filename}")

### load high gamma data so we can do roi analysis on it
once we have more subjects, turn this into a function and loop over all subjects.  
this code is a crime against humanity

In [ ]:
import os
import mne
import numpy as np


def load_mne_objects(sub, output_name, task, LAB_root=None):
    """
    Load MNE objects for a given subject and output name.

    Parameters:
    - sub (str): Subject identifier.
    - output_name (str): Output name used in the file naming.
    - task (str): Task identifier.
    - LAB_root (str, optional): Root directory for the lab. If None, it will be determined based on the OS.

    Returns:
    A dictionary containing loaded MNE objects.
    """

    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    
    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Define file paths
    HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
    HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
    HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'

    # Load the objects
    HG_ev1 = mne.read_epochs(HG_ev1_file)
    HG_base = mne.read_epochs(HG_base_file)
    HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    return {
        'HG_ev1': HG_ev1,
        'HG_base': HG_base,
        'HG_ev1_rescaled': HG_ev1_rescaled,
        'HG_ev1_evoke': HG_ev1_evoke,
        'HG_ev1_evoke_rescaled': HG_ev1_evoke_rescaled
    }

# Example usage
# sub = 'D0057'
# output_name = "Stimulus_i25and75_fixationCrossBase_1sec_mirror"
# task = 'GlobalLocal'
loaded_objects_D0057_i = load_mne_objects('D0057', "Stimulus_c25_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0057_c = load_mne_objects('D0057', "Stimulus_c75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

# Access the objects
HG_ev1_D0057_i = loaded_objects_D0057_i['HG_ev1']
HG_base_D0057_i = loaded_objects_D0057_i['HG_base']
HG_ev1_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_rescaled']
HG_ev1_evoke_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke_rescaled']

HG_ev1_D0057_c = loaded_objects_D0057_c['HG_ev1']
HG_base_D0057_c = loaded_objects_D0057_c['HG_base']
HG_ev1_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_rescaled']
HG_ev1_evoke_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke_rescaled']

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
168 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.

keep working on this later. Split this apart from the other code, that notebook is way too big.